<a href="https://colab.research.google.com/github/HuanAII/PosTagging/blob/main/PosTagging_UsingPenTreeBank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load Dataset**

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux"

In [2]:
#import library
import numpy as np
import torch
import evaluate
from typing import List
from sklearn.model_selection import train_test_split
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [3]:
tagged_sentences=nltk.corpus.treebank.tagged_sents()
print("Number of samples: ", len(tagged_sentences))
#tao dataset
sentences, sentence_tags=[],[]
for tagged_sentence in tagged_sentences:
  sentence, tagged_sentence = zip(*tagged_sentence)
  sentences.append([word.lower() for word in sentence])
  sentence_tags.append(word.lower() for word in tagged_sentence)

sentences

Number of samples:  3914


[['pierre',
  'vinken',
  ',',
  '61',
  'years',
  'old',
  ',',
  'will',
  'join',
  'the',
  'board',
  'as',
  'a',
  'nonexecutive',
  'director',
  'nov.',
  '29',
  '.'],
 ['mr.',
  'vinken',
  'is',
  'chairman',
  'of',
  'elsevier',
  'n.v.',
  ',',
  'the',
  'dutch',
  'publishing',
  'group',
  '.'],
 ['rudolph',
  'agnew',
  ',',
  '55',
  'years',
  'old',
  'and',
  'former',
  'chairman',
  'of',
  'consolidated',
  'gold',
  'fields',
  'plc',
  ',',
  'was',
  'named',
  '*-1',
  'a',
  'nonexecutive',
  'director',
  'of',
  'this',
  'british',
  'industrial',
  'conglomerate',
  '.'],
 ['a',
  'form',
  'of',
  'asbestos',
  'once',
  'used',
  '*',
  '*',
  'to',
  'make',
  'kent',
  'cigarette',
  'filters',
  'has',
  'caused',
  'a',
  'high',
  'percentage',
  'of',
  'cancer',
  'deaths',
  'among',
  'a',
  'group',
  'of',
  'workers',
  'exposed',
  '*',
  'to',
  'it',
  'more',
  'than',
  '30',
  'years',
  'ago',
  ',',
  'researchers',
  'reported'

In [4]:
#preprocessing
#chia du lieu thanh tap train val test
train_sentences, test_sentences, train_tags, test_tags=train_test_split(sentences, tagged_sentences, test_size=0.3)
valid_sentences, test_sentences, valid_tags, test_tags=train_test_split(test_sentences, test_tags, test_size=0.5)
len(train_tags)

2739

In [11]:
from collections import defaultdict
from transformers import AutoModelForTokenClassification
model_name = "QCRI/bert-base-multilingual-cased-pos-english"

model = AutoModelForTokenClassification.from_pretrained(model_name)
label2id=defaultdict(int, model.config.label2id)
id2label={id:label for label, id in label2id.items()}

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# Import thư viện
from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch.utils.data import Dataset
# Khai báo model và tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Xây dựng Dataset cho POS Tagging
class PosTagging_Dataset(Dataset):
    def __init__(self, sentences: List[List[str]], tags: List[List[str]], tokenizer, label2id, max_length: int = 256):
        self.sentences = sentences
        self.tags = tags
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        words = self.sentences[idx]
        labels = self.tags[idx]

        # Token hóa câu
        encoding = self.tokenizer(words, is_split_into_words=True, padding="max_length",
                                  truncation=True, max_length=self.max_length, return_tensors="pt")

        # Chuyển nhãn sang ID
        label_ids = [self.label2id[label] for label in labels]
        label_ids = label_ids[:self.max_length] + [self.label2id["O"]] * (self.max_length - len(label_ids))

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label_ids, dtype=torch.long),
        }

# Tạo tập dữ liệu
train_dataset = PosTagging_Dataset(train_sentences, train_tags, tokenizer, label2id)
val_dataset = PosTagging_Dataset(valid_sentences, valid_tags, tokenizer, label2id)
test_dataset = PosTagging_Dataset(test_sentences, test_tags, tokenizer, label2id)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
accuracy = evaluate.load("accuracy")
ignoreLabel=len(label2id)


In [14]:
import numpy as np
import evaluate

# Load module đánh giá accuracy
accuracy = evaluate.load("accuracy")

# Xác định nhãn cần bỏ qua
ignore_label = len(label2id)

# Hàm tính độ chính xác
def compute_metrics(eval_pred):
    """
    Tính toán độ chính xác của mô hình POS tagging.

    Parameters:
        eval_pred (tuple): (dự đoán của mô hình, nhãn thực tế)

    Returns:
        dict: Kết quả đánh giá (accuracy)
    """
    # Tách dữ liệu dự đoán và nhãn thực tế
    predictions, labels = eval_pred

    # Tạo mask để bỏ qua các nhãn không hợp lệ (ignore_label)
    valid_mask = labels != ignore_label

    # Chọn nhãn có xác suất cao nhất (argmax)
    predicted_labels = np.argmax(predictions, axis=-1)

    # Lọc các giá trị hợp lệ để tính độ chính xác
    filtered_predictions = predicted_labels[valid_mask]
    filtered_labels = labels[valid_mask]

    # Tính accuracy
    return accuracy.compute(predictions=filtered_predictions, references=filtered_labels)


In [17]:
import os
os.environ["WANDB_DISABLED"]="true"

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-18-53f522d294af>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.000734,1.000000
2,No log,0.000516,1.000000
3,0.178200,0.000407,1.000000
4,0.178200,0.000337,1.000000
5,0.178200,0.000288,1.000000
6,0.000600,0.000254,1.000000
7,0.000600,0.000230,1.000000
8,0.000600,0.000214,1.000000
9,0.000400,0.000204,1.000000
10,0.000400,0.000201,1.000000


TrainOutput(global_step=1720, training_loss=0.05215136183122563, metrics={'train_runtime': 1951.131, 'train_samples_per_second': 14.038, 'train_steps_per_second': 0.882, 'total_flos': 3579882599208960.0, 'train_loss': 0.05215136183122563, 'epoch': 10.0})

In [ ]:
# Sau quá trình huấn luyện và mô hình có độ đo đánh giá tốt nhất được sử dụng để gán nhãn cho các
# Câu đầu vào
# tokenization
test_sentence = "We are exploring the topic of deep learning"
input = torch.as_tensor([tokenizer.convert_tokens_to_ids(test_sentence.split())])
input = input.to("cuda")

# prediction
outputs = model(input)
_, preds = torch.max(outputs.logits,-1)
preds = preds[0].cpu().numpy()

# decode
pred_tags = ""
for pred in preds:

pred_tags += id2label[pred] + " "
pred_tags # => PRP VBP RB DT NN IN JJ NN